# Simulación de un sistema con gravedad.

Utilizando el motor de física, que realiza todos los calculos, Pymunk y graficando con Bqplot 
realizamos armamos un ambiente de experimentación para sistemas de cinemática y dinámica clásica ( Newtoniana ).

In [8]:
import pymunk
import numpy as np
from bqplot import Figure, Axis, Scatter, LinearScale,Lines
import time
import bqplot.pyplot as plt
import ipywidgets as widgets

### Seteo del experimento en Pymunk

Creamos un espacio (space), una pelota ( body-shape ) y un piso (line).

In [9]:
space = pymunk.Space()      # Create a Space which contain the simulation
space.gravity = 0,-1000     # Set its gravity

body = pymunk.Body(1,1666)  # Create a Body with mass and moment
body.position = 50,100      # Set the position of the body


shape = pymunk.Circle(body, 5, (0, 0))
shape.elasticity = 0.95
shape.friction = 0.9

space.add(body, shape)       # Add both body and shape to the simulation

line = pymunk.Segment(space.static_body, (0, 0), (100, 0), 0.0)
line.elasticity = 0.95
line.friction = 0.9
space.add(line)

### Armado de la figura 

Con la pelota ( scatter ) y el piso ( floor )

In [14]:
x_sc = LinearScale(min=0,max=100)
y_sc = LinearScale(min=0,max=100)

ax = Axis(label='x',scale=x_sc)
ay = Axis(label='y',scale=y_sc,orientation='vertical')

scatter = Scatter(x=[body.position.x],
                  y=[body.position.y],
                  scales={'x': x_sc, 'y': y_sc, 'size':LinearScale(min=0,max=1)},
                  size=np.array([5]))

floor = Lines(x=[0,100],y=[0,0],scales={'x': x_sc, 'y': y_sc, 'size':LinearScale(min=1,max=10)})
fig = Figure(axes=[ax,ay],marks=[scatter,floor],animation_duration=int(100/6))

### Creamos widgets para interactividad

In [19]:
play_button = widgets.ToggleButton(
    value=False,
    description='Play',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)    

enable_change = widgets.Checkbox(
    value=False,
    description='Enable position change',
    disabled=False
)

duration = widgets.IntText(
    value=100,
    description='Pasos de simulación:',
    disabled=False
)

ball_size = widgets.IntSlider(
    value=7,
    min=1,
    max=10,
    step=1,
    description='Size',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

ball_elasticiy = widgets.FloatSlider(
    value=shape.elasticity,
    min=0,
    max=1.0,
    step=0.1,
    description='Ball_elasticity:',
    disabled=False,
    continuous_update=False,
    readout=True,
    readout_format='.1f',
)

def update_posistion(change):
    if enable_change.value:
        body.position = scatter.x[0], scatter.y[0]
        enable_change.value = False
        scatter.enable_move = False

scatter.observe(update_posistion, names='x')
scatter.observe(update_posistion, names='y')



def run(change):
    enable_change.value = False
    play_button.description = 'Pause'
    play_button.icon = 'pause'
    initial_time = trajectory.x[-1] + 1 if len(trajectory.x) else 0
    for i in range(duration.value):
        space.step(1/60/2)
        scatter.x = [body.position.x]
        scatter.y = [body.position.y]
        trajectory.x = np.append(trajectory.x, initial_time + i )
        trajectory.y = np.append(trajectory.y, scatter.y ) 
        time.sleep(1/100)
        
    play_button.description = 'Play'
    play_button.icon = 'play'
        
def enable_move(change):
    scatter.enable_move = change.new
    trajectory.x = np.array([])
    trajectory.y = np.array([])
    
play_button.observe(run,names='value')
enable_change.observe(enable_move,names='value')

def size_change(change):
    scatter.size = [change.new]

ball_size.observe(size_change,names='value')

def update_elasticity(change):
    shape.elasticity = change.new

ball_elasticiy.observe(update_elasticity,names='value')

### Creamos un grafico adicional que grafica la posición del centro de la pelota en función del tiempo 

In [23]:
t_sc = LinearScale(min=0,max=1000)

ax_t = Axis(label='t', scale=t_sc, tick_format='0.0f')
ax_y = Axis(label='y', scale=y_sc,
            orientation='vertical', tick_format='0.2f')


trajectory = Lines(x=np.array([]),y=np.array([]),scales={'x':t_sc,'y':y_sc})
trajectory_fig = Figure(axes=[ax_t,ax_y],marks=[trajectory],
                        animation_duration=int(100/6),
#                         layout={'width':'500px'}
                       )

In [24]:
widgets.HBox( [  widgets.VBox([fig, enable_change, 
                                  duration, 
                                  play_button,
                                  ball_size, 
                                  ball_elasticiy,
                                  ]),  trajectory_fig] )